In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn.init

In [3]:
cifar10_train = dsets.CIFAR10(root = 'E:\cifar10', train = True, transform = transforms.ToTensor(), download = True)
cifar10_test = dsets.CIFAR10(root = 'E:\cifar10', train = False, transform = transforms.ToTensor(), download = True)

Extracting E:\cifar10\cifar-10-python.tar.gz to E:\cifar10
Files already downloaded and verified


In [4]:
batch_size = 128
lr = 0.001
epochs = 2

In [5]:
device = 'cuda' if torch.cuda.is_available else 'cuda'
train_loader = DataLoader(dataset = cifar10_train, batch_size = batch_size)
test_loader = DataLoader(dataset = cifar10_test, batch_size=4)

In [17]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3,6,5) # input channel, output channel, kernel
        self.pool = nn.MaxPool2d(2,2) # kernel size, stride
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120) # input features, output features
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [18]:
model = net.to(device)
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr)

In [21]:
total_loader = len(train_loader)

for epoch in range(epochs):
    avg_loss = 0
    for image, label in train_loader:
        image = image.to(device)
        label = label.to(device)
        
        optimizer.zero_grad()
        out = model(image)
        loss = criterion(out, label)
        loss.backward()
        optimizer.step()
        
        avg_loss += loss / total_loader
    print('[Epoch: %d], loss = %4f' % (epoch + 1, avg_loss))

[Epoch: 1], loss = 1.908391
[Epoch: 2], loss = 1.614249


In [28]:
import pandas as pd
with torch.no_grad():
    for test_image, test_label in test_loader:
        test_image = test_image.to(device)
        test_label = test_label.to(device)
        
        out = model(test_image)
        correct_prediction = torch.argmax(out, 1) == test_label
        acc = correct_prediction.float().mean()
    print('Accuracy : ', acc.item())
    
df = pd.DataFrame([acc.item()], columns = ['Accuracy'])
df.to_csv('E:\cifar10/acc.csv')

Accuracy :  0.75
